In [ ]:
# 2023-05-23 - This code was created to quickly backup all of the good `loadedSessPickle.pkl` files from the session folders into the './outputs' subfolder in the session:
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/outputs/loadedSessPickle.pkl already in outputs folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/outputs/loadedSessPickle.pkl already in outputs folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/outputs/loadedSessPickle.pkl already in outputs folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/outputs/loadedSessPickle.pkl already in outputs folder.
# p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/outputs/loadedSessPickle.pkl already in outputs folder.
# ...

In [7]:
import os
from pathlib import Path
# import glob # for finding .whl file after building binary repo

In [ ]:
./KDIBA/vvp01/one/2006-4-18_13-6-1/loadedSessPickle.pkl

In [ ]:
# Use glob to find the first generated .whl file in the dist/ directory
found_whl_files = glob.glob('*.whl')
found_whl_files = [a for a in found_whl_files if not a.endswith('current.whl')] # exclude the symlink from the search

In [ ]:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'global_computation_results.pkl'

In [20]:
## Moves all previously saved loadedSessPickle.pkl files to the outputs folder.
correctly_placed_output_files = []
search_path = Path("/home/halechr/turbo/Data/KDIBA/").resolve()
assert search_path.exists()
for p in search_path.rglob(completed_pipeline_filename):
    if p.parent.name != 'outputs':
        # If not already in the 'outputs/' subfolder, move it there.
        print(f'p: {p}')
        curr_output_dir = p.parent.joinpath('outputs')
        curr_output_dir.mkdir(exist_ok=True)
        # print(f'curr_output_dir: {curr_output_dir}')
        new_path = p.replace(curr_output_dir.joinpath(p.name))
        print(f'\t new_path: {new_path}')
        correctly_placed_output_files.append(p)
    else:
        print(f'p: {p} already in outputs folder.')
        correctly_placed_output_files.append(p)
    
correctly_placed_output_files

p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-01_12-58-54/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-05_19-26-43/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_11-43-50/outputs/loadedSessPickle.pkl already in outputs folder.
p: /nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-02_19-28-0/out

[PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_12-3-25/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_21-17-16/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-04_21-20-3/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-03_21-26-8/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_22-4-5/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/fet11-01_12-58-54/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-05_19-26-43/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-09_11-43-50/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/pin01/one/11-02_19-28-0/outputs/loadedSessPickle.pkl'),
 PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/one/2006-4-18_13-6-1/outputs/lo